In [1]:
import pandas as pd
import os
import json
import numpy as np
from datetime import datetime
from collections import Counter
from Code.UtilityFunctions.run_query import run_query

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

___
___
___
# **DO NOT RUN ALL LINES BELOW. SOME YELP FILES ARE VERY LARGE, AND YOU MIGHT RUN OUT OF RAM.**

In [3]:
business = pd.read_json(file_or_buf=os.path.join(read_dir, 'yelp_academic_dataset_business.json'), lines=True)
business['categories'] = business['categories'].str.split(', ', expand=False)

In [4]:
users = pd.read_json(file_or_buf=os.path.join(read_dir, 'yelp_academic_dataset_user.json'), lines=True)

In [ ]:
reviews = pd.read_json(file_or_buf=os.path.join(read_dir, 'yelp_academic_dataset_review.json'), lines=True)
#reviews = pd.merge(left=reviews, right=business, how='left', left_on='business_id', right_on='business_id')
#reviews = pd.merge(left=reviews, right=users, how='left', left_on='user_id', right_on='user_id')

In [10]:
checkins = pd.read_json(file_or_buf=os.path.join(read_dir, 'yelp_academic_dataset_checkin.json'), lines=True)
checkins['date'] = checkins['date'].str.split(', ', expand=False)
checkins = checkins.explode('date')

In [ ]:
tips = pd.read_json(file_or_buf=os.path.join(read_dir, 'yelp_academic_dataset_tip.json'), lines=True)

___
___
___
### CQ 1: How many different types of businesses are defined in Yelp?

In [8]:
sparql_query ="""

SELECT COUNT(DISTINCT(?category))
WHERE {
    ?business schema:keywords ?category .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,1311


In [ ]:
unique_categories = {category for sublist in business['categories'] if sublist for category in sublist}
len(unique_categories)

1311

### CQ 2: How many businesses of type "Restaurants" exist?

In [ ]:
sparql_query = """

SELECT COUNT(DISTINCT(?business)) AS ?numberRestaurants
WHERE {
    ?business schema:keywords yelpcat:Restaurants .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,numberRestaurants.value
0,52268


In [ ]:
counter = 0
for row in business['categories']:
    if row is not None and "Restaurants" in row:
        counter += 1
print(counter)

52268


### CQ 3: How many businesses of type Restaurants have been reviewed?

In [ ]:
sparql_query = """
SELECT COUNT(DISTINCT ?business) AS ?numberRestaurants
WHERE {
    ?business schema:keywords yelpcat:Restaurants .
    ?review schema:about ?business .
}
"""

run_query(query=sparql_query, as_dataframe=True)

,numberRestaurants.value
0,52268


In [ ]:
review_unique_business = reviews.drop_duplicates(subset=['business_id'])
mask = review_unique_business['categories'].apply(lambda x: x is not None and "Restaurants" in x)
len(review_unique_business[mask])

52268

### CQ 4: How many businesses have been reviewed?

In [ ]:
sparql_query = """

SELECT COUNT(DISTINCT(?business))
WHERE {
    ?review schema:about ?business .
    ?review rdf:type schema:UserReview .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,150346


In [ ]:
review_unique_business = reviews.drop_duplicates(subset=['business_id'])
len(review_unique_business)

150346

### CQ 5: How many businesses have, on average, a rating above 4?

In [11]:
sparql_query = """
SELECT COUNT(DISTINCT(?business))
WHERE {
    ?business schema:aggregateRating ?rating .
    FILTER (?rating > 4) .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,43488


In [9]:
business[business['stars'] > 4]['stars'].count()

43488

### CQ 6: What is the average rating across businesses?

In [ ]:
sparql_query = """
SELECT AVG(?rating) as ?averagerating
WHERE {
    ?business schema:aggregateRating ?rating .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,averagerating.type,averagerating.datatype,averagerating.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#decimal,3.59672355766033


In [ ]:
business['stars'].mean()

3.5967235576603303

### CQ 7: How many businesses have been reviewed in Santa Barbara, CA?

In [7]:
sparql_query = """
SELECT COUNT(DISTINCT(?s)) AS ?count_business
WHERE {
    ?s yelpvoc:locatedInCity 'Santa Barbara'^^xsd:string.
}
"""

run_query(query=sparql_query, as_dataframe=True)

,count_business.value
0,3829


In [6]:
business_santa_barbara = business[business['city'] == "Santa Barbara"]
len(business_santa_barbara['business_id'])

3829

### CQ 8: What business has the highest number of visists in one day?

In [3]:
sparql_query = """
SELECT ?business ?year ?month ?day SUM(?visit) as ?numberOfVisits
WHERE {
    ?blanknode schema:object ?business .
    ?blanknode schema:startTime ?time .
    ?blanknode schema:interactionStatistic ?visit .
    BIND (day(?time)  as ?day)
    BIND (month(?time) as ?month)
    BIND (year(?time) as ?year)
}
GROUP BY ?business ?year ?month ?day
ORDER BY DESC(SUM(?visit))
LIMIT 1

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,year.value,month.value,day.value,numberOfVisits.value
0,https://purl.archive.org/purl/yelp/yelp_entities#business_id/CySqUcNz8oPiQTu4EXTnig,2016,6,25,465


In [11]:
checkins["Day"] = checkins["date"].apply(lambda x: x.split("-")[2][:2])
checkins["Month"] = checkins["date"].apply(lambda x: x.split("-")[1])
checkins["Year"] = checkins["date"].apply(lambda x: x.split("-")[0])

checkins.value_counts(subset=["business_id", "Day", "Month", "Year"], sort=True, ascending=False).head(1)

business_id             Day  Month  Year
CySqUcNz8oPiQTu4EXTnig  25   06     2016    465
dtype: int64

### CQ 9: How many vists does the most visisted business have?

In [4]:
sparql_query = """
SELECT ?business SUM(?visits) AS ?count_visits
WHERE {
    ?blanknode rdf:type schema:ArriveAction .
    ?blanknode schema:object ?business .
    ?blanknode schema:interactionStatistic ?visits .
}
GROUP BY ?business 
ORDER BY DESC(SUM(?visits))
LIMIT 1

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,count_visits.value
0,https://purl.archive.org/purl/yelp/yelp_entities#business_id/-QI8Qi8XWH3D8y8ethnajA,52144


In [12]:
checkins.value_counts(subset=["business_id"], sort=True, ascending=False).head(1)

business_id           
-QI8Qi8XWH3D8y8ethnajA    52144
dtype: int64

### CQ 10: How many people have written a review on Yelp?

In [9]:
sparql_query = """
SELECT COUNT(DISTINCT(?user)) AS ?countUsers
WHERE {
    ?review schema:author ?user .
}
"""
run_query(query=sparql_query, as_dataframe=True)

,countUsers.value
0,1987929


In [7]:
reviews.drop_duplicates(subset=['user_id']).shape[0]

1987929

### CQ 11: How many users have more than 10 friends?

In [23]:
sparql_query = """
SELECT COUNT(*) as ?usersWith10Friends
WHERE {
    SELECT ?user COUNT(?friend) AS ?countUsers
    WHERE {
        ?user rdf:type schema:Person .
        ?user schema:knows ?friend .
    }
    GROUP BY ?user
    HAVING (COUNT(?friend) > 10)
}
"""

run_query(query=sparql_query, as_dataframe=True)

,usersWith10Friends.value
0,758803


In [21]:
users["amountFriends"] = users["friends"].apply(lambda x: len(x.split(",")))
users[users["amountFriends"] > 10].shape[0]

758803

### CQ 12: How many friends does a user have on average?

In [99]:
sparql_query="""
SELECT (xsd:double(?countFriends) / xsd:double(?countUser) AS ?averageFriends)
WHERE {
    {SELECT (COUNT(?user) AS ?countUser)
    WHERE {
        ?user rdf:type schema:Person .
        }}
    {SELECT (COUNT(?friend) AS ?countFriends)
    WHERE {
        ?user schema:knows ?friend .
        }}
}
"""

user_friend_counts = run_query(sparql_query, as_dataframe=True)

In [100]:
user_friend_counts

,countUser.value,countFriends.value,averageFriends.value
0,1987897,105225474,52.9331


In [42]:
# Open the file containing the user data
with open(file="/home/ubuntu/OneDrive/DVML-P7/Data/yelp_academic_dataset_user.json", mode="r") as file:
    # Initialize an empty list to store the number of friends for each user
    number_of_friends = []
    # Iterate over each line in the file
    for line in file:
        # Parse the data from the line as a JSON object
        data = json.loads(line)
        # Extract the list of friends for the current user
        friend_ids = data['friends']
        # If the user has friends, append the number of friends to the list
        if friend_ids != 'None':
            number_of_friends.append(len(friend_ids.split(', ')))
        # If the user does not have friends, append 0 to the list
        else:
            number_of_friends.append(0)
# Calculate the mean number of friends
mean_num_friends = np.mean(number_of_friends)
# Print the result
print(mean_num_friends)

52.93306142119033

### CQ 13: How many users have authored 10 reviews?

In [84]:
sparql_query = """
SELECT COUNT(DISTINCT(?user)) AS ?countUsers
WHERE {
    SELECT ?user COUNT(?review) as ?numberOfReviews
    WHERE {
        ?user rdf:type schema:Person .
        ?review schema:author ?user .
    }
    GROUP BY ?user
    HAVING (COUNT(?review) = 10)
}
"""

run_query(query=sparql_query, as_dataframe=True)

,countUsers.value
0,14119


In [17]:
reviews.groupby("user_id").size().reset_index(name="count").query("count == 10").shape[0]

14119

### CQ 14: How many reviews did users make in May 2018?

In [12]:
sparql_query = """
SELECT ?year ?month COUNT(?review) as ?countReviews
WHERE {
    ?review rdf:type schema:UserReview .
    ?review schema:dateCreated ?date .
    BIND (month(?date) as ?month) .
    BIND (year(?date) as ?year) .
    VALUES ?year {2018}
    VALUES ?month {5}
}
GROUP BY ?year ?month
"""

run_query(query=sparql_query, as_dataframe=True)

,year.value,month.value,countReviews.value
0,2018,5,79434


In [3]:
reviews['YEAR'] = reviews.date.dt.year
reviews['MONTH'] = reviews.date.dt.month
reviews.query("YEAR == 2018 & MONTH == 5").shape[0]

79434

In [126]:
reviewers = 0
# Open JSON file for reading
with open(file=os.path.join(read_dir, "yelp_academic_dataset_review.json"), mode="r") as file:
    # Iterate through each line in the file
    for line in file:
        # Parse line as a dictionary
        data = json.loads(line)
        review_date = datetime.strptime(data['date'], '%Y-%m-%d %H:%M:%S')
        if review_date.year == 2018 and review_date.month == 5:
            reviewers += 1
reviewers

79434

### CQ 15: How many parking options can a business provide?

In [7]:
sparql_query = """
SELECT COUNT(DISTINCT ?p)
WHERE {
    ?s rdf:type schema:ParkingFacility.
    ?s ?p ?parking .
    MINUS {
        ?s rdfs:Class ?parking.
        }
}
"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,5


In [8]:
# Initialize empty list to store business parking options
BusinessParking = []
# Open JSON file for reading
with open(file=os.path.join(read_dir, "yelp_academic_dataset_business.json"), mode="r") as file:
    # Iterate through each line in the file
    for line in file:
        # Parse line as a dictionary
        data = json.loads(line)
        try:
            # Extract 'BusinessParking' value from dictionary
            _dict = data['attributes']['BusinessParking']
            # If 'BusinessParking' value is a string, modify it and parse as a dictionary
            if isinstance(_dict, str):
                _dict = _dict.replace("'", '"').replace("None", "null").replace('u"', '"').replace("True", "true").replace("False", "false") 
                _dict = json.loads(_dict)
            # Store modified dictionary in 'parkingopt' variable
            parkingopt = _dict
            # Append 'parkingopt' to 'BusinessParking' list
            BusinessParking.append(parkingopt)
        # If any errors are raised (e.g. missing keys), do nothing and continue to next iteration
        except:
            pass


In [9]:
# Initialize empty list to store parking option names
num_parkingopt = []
# Iterate through each business parking option dictionary
for parkings in BusinessParking:
    # If the dictionary is not empty (i.e. is not 'None')
    if parkings is not None:
        # Iterate through each key (i.e. parking option name) in the dictionary
        for parkingopt in parkings.keys():
            # Append the parking option name to the list
            num_parkingopt.append(parkingopt)
# Convert list to a set to remove duplicate values, and assign to 'set(num_parkingopt)'
len(set(num_parkingopt))


5

### CQ 16: How many businesses has karaoke music?

In [89]:
sparql_query = """
SELECT COUNT(DISTINCT ?business) AS ?businessesWithKaraoke
WHERE {
    ?business yelpvoc:hasMusic ?blank .
    ?blank yelpvoc:haskaraoke 1 .
}
"""

run_query(query=sparql_query, as_dataframe=True)

,businessesWithKaraoke.value
0,75


In [13]:
# Initialize a list to store the karaoke values found in the input file
karaoke_values = []

# Open the input file for reading
with open(file=os.path.join(read_dir, "yelp_academic_dataset_business.json"), mode="r") as file:
    # Iterate over each line in the file
    for line in file:
        # Load the JSON data from the line
        data = json.loads(line)
        try:
            # Extract the Music dictionary from the data
            music_dict = data['attributes']['Music']
            # Check if the Music value is a string
            if isinstance(music_dict, str):
                # Replace various substrings in the Music string with their JSON equivalent
                music_dict = music_dict.replace("'", '"').replace("None", "null").replace('u"', '"').replace("True", "true").replace("False", "false") 
                # Parse the Music string as JSON
                music_dict = json.loads(music_dict)
            # Extract the karaoke value from the Music dictionary
            karaoke = music_dict['karaoke']
            # Add the karaoke value to the list
            karaoke_values.append(karaoke)
        except:
            # Catch any exceptions that may be raised and do nothing
            pass
# Count the number of True values in the list
dict(Counter(karaoke_values))[True]

75